In [2]:
import sys
SAVE = "/saved/"    #In this folder model snapshot are saved
LIB = "/"           #This contains the IIC-SimCLR implementation files
DATA = "/datasets/MNIST/" #In this folder datasets will be downloaded
sys.path.append(LIB)

In [5]:
import wandb
import torch
from collections import namedtuple
from torchvision.datasets import CIFAR10
from torchvision.datasets import STL10
from torchvision.datasets import MNIST
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from simclr_clustering import *
from simclr_clustering.data.transforms import *
from simclr_clustering.data.cluster_eval import *
from simclr_clustering.data.eval_metrics import *
from simclr_clustering.data.make_data import *
from simclr_clustering.archs import ClusterNetMulHead
from simclr_clustering.archs import IICLoss, NTXentLoss
from simclr_clustering.pipeline import IIC_Pipeline

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Running on:  Tesla T4


In [6]:
config = {
    "output_k_A": 70,
    "output_k_B": 10,
    "output_k_C": 128,
    "output_back": 512,
    "gt_k": 10, #grond truth classes
    "separate_augs" : False;
    "input_sz": 32,
    "rand_crop_sz": 20,
    "rot_val": 0,
    "fluid_warp": False,
    "crop_orig": True,
    "multiple_crop": False,
    "crop_szs": [16,20,24],
    "cutout": False,
    "sobel": True,
    "cutout_p": False,
    "cutout_max_box": False,
    "demean": False,
    "per_img_demean": False,
    "simclr_bs": 512,
    "iic_dataloader_bs": 220, # iic_bs / num_dataloaders
    "iic_bs": 660,
    "batchnorm_track" : True,
    "include_rgb" : False,
    "using_ir" : False,
    "in_channels" : 2,
    "num_sub_heads" : 5,
    "num_dataloaders": 3,
    "dataset": 'CIFAR10',
    "dataset_root": DATA,
    "train_partitions_head_A" : [True, False],
    "train_partitions_head_B" : [True, False],
    "mapping_assignment_partitions" : [True, False],
    "mapping_test_partitions" : [True, False],
    "eval_mode": 'hung', #orig for original, hung for hungarian
    "mode": 'IID',
    "test_code": False,
    "num_epochs": 500,
    "device": 'cuda',
    "warmup": 20,
    "lr_iic": 0.0001,
    "lr_simclr": 0.001,
    "lr_gamma_iic": None,
    "lr_milestones_iic": None,
    "num_epochs_head": 1,
}

config_log = {
    'log' : False,
    'project' : 'my-project',
    'run': 'my-run'
}
config_log = namedtuple('Config_l',config_log.keys())(*config_log.values())
config = namedtuple('Config',config.keys())(*config.values())

In [8]:
_ = CIFAR10(config.dataset_root, train=True, download=True)
_ = CIFAR10(config.dataset_root, train=False, download=True)

iic = IIC_Pipeline(config)
#uncomment next line to load a previously saved checkpoint
#iic.load_checkpoint(path=SAVE+'model.pth')
iic.exec_pipeline(config_log, save_path=SAVE+'model.pth', do_clustering=True, do_overclustering=True, do_simclr=True)

Running on:  Tesla T4
ClusterNet ready
adding rotation option for imgs_tf: 25
Dataloaders ready


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jaimec (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20200912_075614-2gylds7m
wandb: Syncing run mnist



Starting training-evalutation loop: 
 Epoch  331
  IIC overclustering loss:  -3.5798264596155818
  IIC loss:  -2.11945928077339
  SimCLR loss:  5.02616619011935


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


  Average acc:  0.7755
  Best acc:  0.8611571
 Epoch  332
  IIC overclustering loss:  -3.5764307019105153
  IIC loss:  -2.1184833281466204
  SimCLR loss:  5.024922448046067
  Average acc:  0.7732457
  Best acc:  0.85907143
 Epoch  333
  IIC overclustering loss:  -3.5848433844348104
  IIC loss:  -2.119027971100284
  SimCLR loss:  5.026087666259093
  Average acc:  0.7671657
  Best acc:  0.8591
 Epoch  334
  IIC overclustering loss:  -3.5829262367236576
  IIC loss:  -2.119200788695237
  SimCLR loss:  5.024400732096503
  Average acc:  0.7732743
  Best acc:  0.86394286
 Epoch  335
  IIC overclustering loss:  -3.5861839343761575
  IIC loss:  -2.119841169040405
  SimCLR loss:  5.0242170200628395
  Average acc:  0.76693714
  Best acc:  0.85344285
 Epoch  336
  IIC overclustering loss:  -3.5762641982972436
  IIC loss:  -2.1197983942061756
  SimCLR loss:  5.025327801704407
  Average acc:  0.76594573
  Best acc:  0.8546
 Epoch  337
  IIC overclustering loss:  -3.583651540421393
  IIC loss:  -2.11

KeyboardInterrupt: ignored